In [2]:
import pandas as pd
import os
pd.set_option('display.max_colwidth', None)

# this radar_factory is provided by matplotlib https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html
# we will use this to present our compatibilities and skill scores
import radar_factory as rf
import matplotlib.pyplot as plt

import numpy as np
rng = np.random.default_rng() # For testing


Welcome to the prototype! Let's figure stuff out.

In [3]:
# File names and extensions setup. As each file has multiple sheets, we need to track those too.
# If we're using them. We might not, but for now just have the major files.
# header = 1 so we have the titles conveniently, as the first row is the sheet title.

# Aggregate economy
doc_agg19 = pd.read_excel("project_data/2019-29/aggregate-economy-tables.xlsx", sheet_name=None, header = 0)
doc_agg23 = pd.read_excel("project_data/2023-33/aggregate-economy-tables.xlsx", sheet_name=None, header = 0)

# Education
doc_edu19 = pd.read_excel("project_data/2019-29/education.xlsx", sheet_name=None, header = 1)
doc_edu23 = pd.read_excel("project_data/2023-33/education.xlsx", sheet_name=None, header = 1)

# industry
doc_ind19 = pd.read_excel("project_data/2019-29/industry.xlsx", sheet_name=None, header = 1)
doc_ind23 = pd.read_excel("project_data/2023-33/industry.xlsx", sheet_name=None, header = 1)

# labor force
doc_lab19 = pd.read_excel("project_data/2019-29/labor-force.xlsx", sheet_name=None, header = 1)
doc_lab23 = pd.read_excel("project_data/2023-33/labor-force.xlsx", sheet_name=None, header = 1)

# occupation
doc_occ19 = pd.read_excel("project_data/2019-29/occupation.xlsx", sheet_name=None, header = 1)
doc_occ23 = pd.read_excel("project_data/2023-33/occupation.xlsx", sheet_name=None, header = 1)

# skills (just 2023)
skills = pd.read_excel("project_data/2023-33/skills.xlsx", sheet_name=None, header = 1)

In [4]:
# File checking. Still need to do cleanup, but working on it.
sheet_names = list(skills.keys())  # Convert keys to a list
sheet_cats = skills[sheet_names[0]]  # Access the sheets
sheet_cats.head(4)  # Display the first few rows

,Table 6.1 Skills data by major occupational group
0,Table 6.2 Skills data by detailed occupation
1,Table 6.3 Top skills categories for fastest growing occupations
2,Table 6.4 Skills data by typical education needed for entry
3,Table 6.5 Percentile rank of detailed occupations by skills category


In [5]:
# Data by major occupational group
data_by_occ = skills[sheet_names[1]]

In [5]:
data_by_occ.head(1)

,2023 National Employment Matrix title,2023 National Employment Matrix code,"Employment, 2023","Employment, 2033","Employment change, numeric, 2023–33","Employment change, percent, 2023–33",Adaptability,Computers and information technology,Creativity and innovation,Critical and analytical thinking,...,Interpersonal,Leadership,Mathematics,Mechanical,Physical strength and stamina,Problem solving and decision making,Project management,Science,Speaking and listening,Writing and reading
0,"Total, all occupations",00-0000,167849.8,174589.0,6739.2,4.0,3.8,3.0,2.7,3.1,...,3.5,3.2,2.6,1.9,2.2,3.5,2.7,1.7,3.2,3.3


In [6]:
# Truncate columns [1 to 5]
data_by_occ = data_by_occ.iloc[:, [0] + list(range(6, data_by_occ.shape[1]))]

# Now left with jobs and skills. Each skill is scored from 1 to 5, least important to extremely important.
# We use these values as a positive multiplier for the final score.

data_by_occ.head()

,2023 National Employment Matrix title,Adaptability,Computers and information technology,Creativity and innovation,Critical and analytical thinking,Customer service,Detail oriented,Fine motor,Interpersonal,Leadership,Mathematics,Mechanical,Physical strength and stamina,Problem solving and decision making,Project management,Science,Speaking and listening,Writing and reading
0,"Total, all occupations",3.8,3.0,2.7,3.1,3.3,3.6,2.5,3.5,3.2,2.6,1.9,2.2,3.5,2.7,1.7,3.2,3.3
1,Management occupations,4.1,3.5,3.0,3.6,3.5,3.6,1.9,3.7,4.0,3.1,1.7,1.7,4.0,3.5,1.7,3.6,3.8
2,Business and financial operations occupations,4.0,3.7,2.8,3.7,3.3,3.6,1.7,3.5,3.4,3.2,1.3,1.3,3.8,3.1,1.4,3.6,3.9
3,Computer and mathematical occupations,3.9,4.3,3.1,3.9,2.6,3.7,2.2,3.1,3.1,3.1,1.9,1.3,3.8,2.7,1.7,3.4,3.8
4,Architecture and engineering occupations,3.8,3.8,3.1,3.8,2.7,3.8,2.2,3.0,3.2,3.6,2.1,1.5,3.8,2.9,2.6,3.3,3.7


Let's get a prototype going to show what we want to do. We essentially want to rate a person's compatibility to an industry based on their skills, and we want to visualize this nicely in some kind of final product. 

In [7]:
# Pulling all the skills
skill_names = data_by_occ.columns[1:].tolist()
skill_names[:4]

['Adaptability',
 'Computers and information technology',
 'Creativity and innovation',
 'Critical and analytical thinking']

In [8]:
# Pulling all the major job categories names
# Truncate combined data and ending notes.
group_names = data_by_occ.iloc[1:-4, 0].tolist()
group_names[:4]


['  Management occupations',
 '  Business and financial operations occupations',
 '  Computer and mathematical occupations',
 '  Architecture and engineering occupations']

In [9]:
data_by_occ.columns[0]

'2023 National Employment Matrix title'

In [10]:
test = data_by_occ[data_by_occ[data_by_occ.columns[0]].str.contains(group_names[0])]
test.iloc[0].tolist()
#test.iloc[0].tolist()[1]

['  Management occupations',
 4.1,
 3.5,
 3.0,
 3.6,
 3.5,
 3.6,
 1.9,
 3.7,
 4.0,
 3.1,
 1.7,
 1.7,
 4.0,
 3.5,
 1.7,
 3.6,
 3.8]

These are the major skills we are working with and are looking to evaluate from the questions. For now, we'll randomly generate the skills for a person, essentially generating a random person that took our quiz!

What is most important is that we will be using the data from the skills sheet to provide a strong weight for a person's individual characteristics, allowing us to provide an accurate score for their compatibility to an industry or even job. For now, we're focusing on the major, categorical parts of the job.

# Done by Nikhil And Alwin

In [11]:
file_path = 'project_data/2023-33/skills.xlsx'

xls = pd.ExcelFile(file_path)

# Step 2: Load the sheet data (Table 6.1) for analysis
sheet_data = pd.read_excel(xls, sheet_name='Table 6.1')

# Step 3: Clean the data
# Dropping the first row (header) and setting correct column names
data_cleaned = sheet_data.drop(0)
data_cleaned.columns = sheet_data.iloc[0]
data_cleaned = data_cleaned.drop(1)

# Step 4: Selecting relevant columns for radar chart: skill columns (starting from column 6)
skills_columns = data_cleaned.columns[6:22]

# Step 5: Select a few rows (occupational groups) to visualize
occupational_groups = data_cleaned['2023 National Employment Matrix title'].values[1:4]
skill_data = data_cleaned[skills_columns].iloc[1:4].values.astype(float)



In [12]:
#========================Edcutation=========================
file_2019_new_path = 'project_data/2019-29/education.xlsx'
file_2023_path = 'project_data/2023-33/education.xlsx'

xls_2019_new = pd.ExcelFile(file_2019_new_path)

# Load the relevant sheets for the 2019 data
df1_2019_new = pd.read_excel(xls_2019_new, sheet_name='Table 5.2')
df1_2019_new_cleaned = df1_2019_new.iloc[2:, [0, 1, 3]].reset_index(drop=True)
df1_2019_new_cleaned.columns = ['Education Level', 'Employment 2019', 'Employment Change 2019-29']
df1_2019_new_cleaned['Employment 2019'] = pd.to_numeric(df1_2019_new_cleaned['Employment 2019'], errors='coerce')

# Clean the 2019 data by removing rows with irrelevant information
df1_2019_new_cleaned = df1_2019_new_cleaned[~df1_2019_new_cleaned['Education Level'].str.contains("Total|Occupation", na=False)]
df1_2019_new_cleaned = df1_2019_new_cleaned.dropna(subset=['Employment 2019'])

# Load the 2023 data (already cleaned)
xls_2023 = pd.ExcelFile(file_2023_path)

# Load the relevant sheets for the 2023 data
df1_2023 = pd.read_excel(xls_2023, sheet_name='Table 5.2')
df1_2023_cleaned = df1_2023.iloc[2:, [0, 1, 3]].reset_index(drop=True)
df1_2023_cleaned.columns = ['Education Level', 'Employment 2023', 'Employment Change 2023-33']
df1_2023_cleaned['Employment 2023'] = pd.to_numeric(df1_2023_cleaned['Employment 2023'], errors='coerce')

# Clean the 2023 data by removing rows with irrelevant information
df1_2023_cleaned = df1_2023_cleaned[~df1_2023_cleaned['Education Level'].str.contains("Total|Occupation", na=False)]
df1_2023_cleaned = df1_2023_cleaned.dropna(subset=['Employment 2023'])

# Data preparation for the radar chart
categories = df1_2019_new_cleaned['Education Level'].tolist()
values_2019 = df1_2019_new_cleaned['Employment 2019'].tolist()
values_2023 = df1_2023_cleaned['Employment 2023'].tolist()

# Normalize the values to a 0-1 scale
max_value = max(max(values_2019), max(values_2023))
values_2019_scaled = [v / max_value for v in values_2019]
values_2023_scaled = [v / max_value for v in values_2023]




#==================Mean Wage==========================
file_2019 = 'project_data/2019-29/oesm19nat/national_M2019_dl.xlsx'  # Use the correct path to your file
file_2023 = 'project_data/2023-33/oesm23nat/national_M2023_dl.xlsx'  # Use the correct path to your file

df_2019 = pd.read_excel(file_2019, sheet_name='national_M2019_dl')
df_2023 = pd.read_excel(file_2023, sheet_name='national_M2023_dl')

# Renaming columns for consistency
df_2019 = df_2019.rename(columns={'occ_code': 'occ_code', 'occ_title': 'occ_title', 'a_median': 'a_median_2019'})
df_2023 = df_2023.rename(columns={'OCC_CODE': 'occ_code', 'OCC_TITLE': 'occ_title', 'A_MEDIAN': 'a_median_2023'})

# Merging the two datasets on occupation code
merged_wages = pd.merge(df_2019[['occ_code', 'occ_title', 'a_median_2019']], 
                        df_2023[['occ_code', 'occ_title', 'a_median_2023']], 
                        on='occ_code', 
                        how='inner')

# Selecting one occ_title column (either occ_title_x or occ_title_y) since they are identical
merged_wages['occ_title'] = merged_wages['occ_title_x']  # You can also choose occ_title_y

# Extended occupations data (adding extra occupations like Software Developers, Registered Nurses, etc.)
extended_occupations = merged_wages[merged_wages['occ_title'].isin([
    'All Occupations', 'Management Occupations', 'Top Executives', 'Chief Executives',
    'Software Developers', 'Registered Nurses', 'Construction Laborers', 'Teachers'
])]
#merged_wages['occ_title'], a medan 2029, 2023

#=========================Skills Rador=======================
file_path = 'project_data/2023-33/skills.xlsx'

xls = pd.ExcelFile(file_path)

# Step 2: Load the sheet data (Table 6.1) for analysis
sheet_data = pd.read_excel(xls, sheet_name='Table 6.1')

# Step 3: Clean the data
# Dropping the first row (header) and setting correct column names
data_cleaned = sheet_data.drop(0)
data_cleaned.columns = sheet_data.iloc[0]
data_cleaned = data_cleaned.drop(1)

# Step 4: Selecting relevant columns for radar chart: skill columns (starting from column 6)
skills_columns = data_cleaned.columns[6:22]

# Step 5: Select a few rows (occupational groups) to visualize
occupational_groups = data_cleaned['2023 National Employment Matrix title'].values[1:4]
skill_data = data_cleaned[skills_columns].iloc[1:4].values.astype(float)



In [13]:
import asyncio
import websockets
import nest_asyncio
import json
from IPython import get_ipython

# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()

# Initialize a global variable for person scores
person_scores = None  # This will hold the extracted ratings from the JSON



# WebSocket listener function
async def websocket_client():
    uri = "ws://localhost:3000"  # WebSocket server URL
    try:
        async with websockets.connect(uri) as websocket:
            print("Connected to WebSocket server. Listening for messages...")
            
            while True:
                # Wait for a message from the server
                response = await websocket.recv()
                print(f"Message received from server: {response}")
                
                # Parse JSON response and extract ratings
                global person
                try:
                    data = json.loads(response)  # Parse the JSON string
                    # Generating a random person with certain skill affinities from 0 to 1
                    # Not yet taking into account the degree yet. That's a bit finicky, and we have to think about
                    # the implications of the value of a degree first. Some jobs value higher degrees whereas others
                    # don't care.
                    person = [entry['rating']/10 for entry in data]  # Extract ratings
                    print(f"Updated person: {person}")
                    person_scores = np.round(np.zeros(len(group_names)), 2)
                    
                    # Scoring a person into each major occupation
                    # Take skill * weight * 20. That's their weighted score for one skill.
                    # At worst, we get 0. At best, we get 100. We will aggregate these scores later.
                    
                    # For each job
                    
                    for i in range(len(group_names)):
                        row = data_by_occ[data_by_occ[data_by_occ.columns[0]].str.contains(group_names[i])]
                        skillweight = row.iloc[0].tolist()
                        # For each skill
                        for j in range(len(skill_names)):
                            score = person[j] * skillweight[j + 1] * 20
                            person_scores[i] += score          


                    perfect_scores = np.zeros(len(group_names))

                    for i in range(len(group_names)):
                        row = data_by_occ[data_by_occ[data_by_occ.columns[0]].str.contains(group_names[i])]
                        skillweight = row.iloc[0].tolist()
                        # For each skill
                        for j in range(len(skill_names)):
                            score = 1 * skillweight[j + 1] * 20
                            perfect_scores[i] += score

                    final_scores = np.zeros(len(group_names))
                    for i in range(len(final_scores)):
                        final_scores[i] = person_scores[i]/perfect_scores[i]
                    final_scores  
                    final_scores_json = json.dumps({"final_scores": final_scores.tolist(),
                                                    "edu_categories": categories,
                                                    "edu_values_2019_scaled" : values_2019_scaled,
                                                    "edu_values_2023_scaled" : values_2023_scaled,
                                                    "occ_title": extended_occupations['occ_title'].tolist(),
                                                    "a_median_2019": extended_occupations['a_median_2019'].tolist(),
                                                    "a_median_2023": extended_occupations['a_median_2023'].tolist(),
                                                    "occupational_groups": occupational_groups.tolist(),
                                                    "skill_data": skill_data.tolist(),
                                                    "skill_column": skills_columns.tolist()
                                                    })  # Convert to JSON
                    await websocket.send(final_scores_json)
                    print(f"Final scores sent to server: {final_scores}")

                except (json.JSONDecodeError, KeyError, TypeError) as e:
                    print(f"Error parsing response: {e}. Expected a JSON array with 'question' and 'rating' fields.")
                    continue

    except Exception as e:
        print(f"Error: {e}")

# Start the WebSocket client
await websocket_client()


Connected to WebSocket server. Listening for messages...
Message received from server: [{"question":"How effectively do you adapt your methods when unexpected changes occur in your work environment?","rating":8},{"question":"How comfortable are you with learning and applying new software tools or technologies?","rating":10},{"question":"How skilled are you at generating new ideas to overcome challenges?","rating":7},{"question":"How effective are you at breaking down complex problems into manageable parts to find a solution?","rating":8},{"question":"How empathetic and patient are you when dealing with frustrated customers?","rating":3},{"question":"How meticulous are you when it comes to catching small errors in your work?","rating":0},{"question":"How comfortable are you performing tasks that require fine motor skills or precise coordination?","rating":7},{"question":"How effectively do you collaborate with team members who have different working styles or personalities?","rating":8}